In [1]:
from yahoo_fin.stock_info import get_data

In [5]:
ticker_list = ["amzn", "aapl", "ba"]
historical_datas = {}
for ticker in ticker_list:
    historical_datas[ticker] = get_data(ticker)

In [13]:
# tickers_dow() returns a list of all the tickers in the Dow Jones:
# tickers_sp500()
# tickers_dow()
# tickers_nasdaq()
import yahoo_fin.stock_info as si
dow_list = si.tickers_dow()

In [34]:
quote_table = si.get_quote_table("aapl", dict_result=True)

In [36]:
quote_table['Quote Price']

127.3499984741211

In [40]:
x = si.get_stats_valuation("aapl")

In [38]:
x

,0,1
0,Market Cap (intraday) 5,2.13T
1,Enterprise Value 3,2.17T
2,Trailing P/E,28.62
3,Forward P/E 1,23.80
4,PEG Ratio (5 yr expected) 1,1.45
5,Price/Sales (ttm),6.53
6,Price/Book (mrq),30.72
7,Enterprise Value/Revenue 3,6.67
8,Enterprise Value/EBITDA 7,21.73


In [41]:
import pandas as pd

In [42]:
income_statement = si.get_income_statement("aapl")
income_statement

endDate,2020-09-26,2019-09-28,2018-09-29,2017-09-30
Breakdown,,,,
researchDevelopment,18752000000,16217000000,14236000000,11581000000
effectOfAccountingCharges,None,None,None,None
incomeBeforeTax,67091000000,65737000000,72903000000,64089000000
minorityInterest,None,None,None,None
netIncome,57411000000,55256000000,59531000000,48351000000
sellingGeneralAdministrative,19916000000,18245000000,16705000000,15261000000
grossProfit,104956000000,98392000000,101839000000,88186000000
ebit,66288000000,63930000000,70898000000,61344000000
operatingIncome,66288000000,63930000000,70898000000,61344000000


In [43]:
transposed = income_statement.transpose()
transposed

Breakdown,researchDevelopment,effectOfAccountingCharges,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,grossProfit,ebit,operatingIncome,otherOperatingExpenses,...,nonRecurring,otherItems,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,discontinuedOperations,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
endDate,,,,,,,,,,,,,,,,,,,,,
2020-09-26,18752000000,None,67091000000,None,57411000000,19916000000,104956000000,66288000000,66288000000,None,...,None,None,9680000000,274515000000,208227000000,169559000000,803000000,None,57411000000,57411000000
2019-09-28,16217000000,None,65737000000,None,55256000000,18245000000,98392000000,63930000000,63930000000,None,...,None,None,10481000000,260174000000,196244000000,161782000000,1807000000,None,55256000000,55256000000
2018-09-29,14236000000,None,72903000000,None,59531000000,16705000000,101839000000,70898000000,70898000000,None,...,None,None,13372000000,265595000000,194697000000,163756000000,2005000000,None,59531000000,59531000000
2017-09-30,11581000000,None,64089000000,None,48351000000,15261000000,88186000000,61344000000,61344000000,None,...,None,None,15738000000,229234000000,167890000000,141048000000,2745000000,None,48351000000,48351000000


In [44]:
balance_sheet = si.get_balance_sheet("aapl")
balance_sheet

endDate,2020-09-26,2019-09-28,2018-09-29,2017-09-30
Breakdown,,,,
totalLiab,258549000000,248028000000,258578000000,241272000000
totalStockholderEquity,65339000000,90488000000,107147000000,134047000000
otherCurrentLiab,47867000000,43242000000,39293000000,38099000000
totalAssets,323888000000,338516000000,365725000000,375319000000
commonStock,50779000000,45174000000,40201000000,35867000000
otherCurrentAssets,11264000000,12352000000,12087000000,13936000000
retainedEarnings,14966000000,45898000000,70400000000,98330000000
otherLiab,46108000000,50503000000,48914000000,43251000000
treasuryStock,-406000000,-584000000,-3454000000,-150000000


In [45]:
cash_flow_statement = si.get_cash_flow("aapl")
cash_flow_statement

endDate,2020-09-26,2019-09-28,2018-09-29,2017-09-30
Breakdown,,,,
investments,5335000000,58093000000,30845000000,-33542000000
changeToLiabilities,-1981000000,-2548000000,9172000000,8373000000
totalCashflowsFromInvestingActivities,-4289000000,45896000000,16066000000,-46446000000
netBorrowings,2499000000,-7819000000,432000000,29014000000
totalCashFromFinancingActivities,-86820000000,-90976000000,-87876000000,-17974000000
changeToOperatingActivities,881000000,-896000000,30016000000,-8480000000
issuanceOfStock,880000000,781000000,669000000,555000000
netIncome,57411000000,55256000000,59531000000,48351000000
changeInCash,-10435000000,24311000000,5624000000,-195000000
